Берём натренированную VGG16, отрезаем голову и добавляем сверху слои для распознавания доски:

In [1]:
cd ..

/home/bfilippov/kaggle/whiteboard


In [2]:
import keras
import pandas as pd
import ast

train = pd.read_csv('source/train.csv')
test = pd.read_csv('source/test.csv')
train['labels'] = train['labels'].map(ast.literal_eval)
test['labels'] = test['labels'].map(ast.literal_eval)

Using TensorFlow backend.


In [3]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf

from keras.layers import Dense, Flatten, Input
from keras import backend as K

from models.model_zoo import whiteboard_detector, whiteboard_loss
from keras.applications import VGG16
from models.image_generator import whiteboard_images

sess = tf.Session()
K.set_session(sess)

input_height = 224
input_width = 224

input_image = Input(shape=(input_height, input_width, 3))
resnet = VGG16(input_tensor=input_image, include_top=False)

for l in resnet.layers:
    l.trainable=False
    
resnet = Flatten()(resnet.output)
resnet = Dense(128, activation='relu')(resnet)

simple_detector = whiteboard_detector(input_image, resnet)

simple_detector.compile(optimizer='adam', loss=whiteboard_loss)


In [ ]:
img_dir = 'source/'
img_size = (input_height, input_width)

imgs = whiteboard_images(train, img_dir, img_size, batch_size=2)

simple_detector.fit_generator(imgs, samples_per_epoch=2000, nb_epoch=10)

Epoch 1/10
 198/2000 [=>............................] - ETA: 2656s - loss: 48711.5142